In [ ]:
import openmc

In this segment, we will go through the basic features of the Python API for constructing input files and running OpenMC. We highly recommend having a copy of the [Python API reference documentation](http://openmc.readthedocs.org/en/latest/pythonapi/index.html) open in another browser tab that you can refer to. Also remember that within a notebook, you can press `Shift+Tab` with the cursor placed over a function, class, or method to see its documentation.

## Defining Materials

We need to define the materials in the simulation.  Let's start with UO2.  Notice the difference between `add_nuclide` and `add_element`.

In [ ]:
#TODO: Define the uo2 Material here.
#TODO: Add 0.03 of 'U235' and 0.97 of 'U238'.
#TODO: Add 2.0 of 'O' and set the density to 10.0 'g/cm3'.

Now, let's define the zirconium and water materials.  To get accurate results with water in a thermal reactor, we need $S(\alpha, \beta)$ scattering tables.

In [ ]:
zirconium = openmc.Material()
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material()
water.add_element('H', 2.0)
water.add_element('O', 1.0)
water.set_density('g/cm3', 0.7)

In [ ]:
#TODO: Add an s_alpha_beta table for 'c_H_in_H2O'

To actually create a materials.xml file, we need to instantiate a `Materials` collection and register our materials with it.

In [ ]:
#TODO: Define Materials and export_to_xml().

## Defining Geometry and Assigning Materials

We start by defining the cylindrical and planar surfaces that we need to create the model.

In [ ]:
#TODO: Define fuel_or with R=0.39 cm.
clad_ir = openmc.ZCylinder(R=0.40)
clad_or = openmc.ZCylinder(R=0.46)

#TODO: Define a pitch of 1.26 cm.
#TODO: Define the left XPlane.
right = openmc.XPlane(x0=pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(y0=-pitch/2, boundary_type='reflective')
top = openmc.YPlane(y0=pitch/2, boundary_type='reflective')

With the surfaces created, we can now take advantage of the built-in operators on surfaces to create regions. The unary `-` and `+` operators correspond to the negative and positive half-spaces of a surface respectively. These half-spaces can then be combined using `&` (intersection), `|` (union), and `~` (complement).

In [ ]:
#TODO: Define the fuel_region and gap_region.
clad_region = +clad_ir & -clad_or
water_region = +left & -right & +bottom & -top & +clad_or

Each cell that we create has to a physical region and an assigned fill (material, universe, or lattice) which is placed in the region.

In [ ]:
#TODO: Define fuel and gap Cell objects.

clad = openmc.Cell()
clad.fill = zirconium
clad.region = clad_region

moderator = openmc.Cell()
moderator.fill = water
moderator.region = water_region

Finally, there is some boilerplate code that we need to assign the cells we created to a universe and tell OpenMC that this universe is the "root" universe.

In [ ]:
#TODO: Define the root Universe.

g = openmc.Geometry()
g.root_universe = root
g.export_to_xml()
!cat geometry.xml

## Geometry plotting

With materials and geometry defined, we can now make a plot of our problem. To create a plot, we simply need to specify the origin and the number of pixels in each direction. By default, the plot will be a "slice" plot which cuts through the geometry. For our case here, we specify that we want the plot to be colored by material (rather than by cell) and we specify the colors to use.

In [ ]:
#TODO: Define the Plot p.

In [ ]:
#TODO: Give it a width of [pitch, pitch].
#TODO: Give it [400, 400] pixels.
#TODO: Set color_by to 'material'
p.colors = {uo2:'salmon', water:'cyan', zirconium:'gray'}

The usual way plots are handled in OpenMC is the following:  Make an `openmc.Plots` object containing the relevant `openmc.Plot` objects.  Call the `openmc.Plots.export_to_xml()` method.  Then call `openmc.plot_geometry`.  That will activate the Fortran plotting module which will then output .ppm plot files.  You can use a utility like `convert` to change the .ppm to .png.

In a Jupyter notebook, you can automate all that with `openmc.plot_inline`.

In [ ]:
openmc.plot_inline(p)

## Starting source and settings

The Python API has a module ``openmc.stats`` with various univariate and multivariate probability distributions. We can use these distributions to create a starting source using the ``openmc.Source`` object.

In [ ]:
point = openmc.stats.Point((0, 0, 0))
src = openmc.Source(space=point)

Now let's create a `Settings` object and give it the source we created along with specifying how many batches and particles we want to run.

In [ ]:
#TODO: Create a Settings object.

In [ ]:
#TODO: Assign the source.
#TODO: Set 100 batches, 10 inactive, 1000 particles.

In [ ]:
settings.export_to_xml()
!cat settings.xml

## User-defined tallies

To give a quick example of how to create tallies, we will show how one would tally the total, fission, absorption, and (n,$\gamma$) reaction rates for $^{235}$U in the cell containing fuel.

In [ ]:
#TODO: Define a Tally t.

In [ ]:
#TODO: Make a CellFilter for the fuel cell.
#TODO: Set the tally filters to [cell_filter].

t.nuclides = ['U235']
t.scores = ['total', 'fission', 'absorption', '(n,gamma)']

Similar to the other files, we need to create a `Tallies` collection, register our tally, and then export it to XML.

In [ ]:
tallies = openmc.Tallies([t])
tallies.export_to_xml()
!cat tallies.xml

## Running OpenMC

Running OpenMC from Python can be done using the `openmc.run()` function. This function allows you to set the number of MPI processes and OpenMP threads, if need be.

In [ ]:
#TODO: Run openmc.

In [ ]:
!cat tallies.out